In [1]:
import numpy as np
import numba as nb
import Ivvsc
import importlib
import time

In [2]:
filename = "T100TTFT.npz"

In [3]:
data = np.load(filename, allow_pickle=True)
settings = data['settings'].item()
raw = data['raw'].item()
prob = data['prob'].item()

In [4]:
N = settings['N'] + 1
e_max = settings['eps_max']
B = 2

p = np.linspace(0, e_max, N)
delta_p = p[1] - p[0]

In [5]:
def calc_c(data, dN, p, e_max, B, dp, output_filename = "data"):
    C_full = np.zeros_like(data)
    
    N_total = len(data)
    
    N_steps = N_total // dN
    
    time_array = np.zeros(N_steps)
    
    for i in range(N_steps):
        beg = time.time()
        for j in range(dN):
            index = i * dN + j
            print("Time step {} / {}".format(index+1, N_total))
            C_full[index, :, :] = Ivvsc.C_Ivvsc(data[index,:,:], p, e_max, B, dp)
        np.savez(output_filename + "-" + str(i), C = C_full[:(i+1)*dN,:,:], p = p)
        print("File {}-{}.npz number {} of {}.".format(output_filename, i, i+1, N_steps))
        time_array[i] = (time.time() - beg) 
        time_string = "{:.1f}\% done. total time: {:.2f} hrs ({:.1f} min for last file)".format(100*(index+1)/N_total, np.sum(time_array)/60/60, time_array[i]/60)
        print(time_string)
        
    if index != len(data) - 1:
        i = N_steps
        for k in range(index+1, len(data)):
            C_full[k, :, :] = Ivvsc.C_Ivvsc(data[k,:,:], p, e_max, B, dp)
            
        
    np.savez(output_filename, C = C_full[:(i+1)*dN,:,:], p = p)
            
    return

In [ ]:
%%time
calc_c(raw['nu3D'], 75, p, e_max, B, delta_p, output_filename="test")

Time step 1 / 759
